# P3D Web notebook
<u>Version</u> : 0.1 (02/06/2021)

In [3]:
from IPython.display import HTML
"""
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code of P3D Web is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')
"""

'\nHTML(\'\'\'<script>\ncode_show=true; \nfunction code_toggle() {\n if (code_show){\n $(\'div.input\').hide();\n } else {\n $(\'div.input\').show();\n }\n code_show = !code_show\n} \n$( document ).ready(code_toggle);\n</script>\nThe raw code of P3D Web is by default hidden for easier reading.\nTo toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.\'\'\')\n'

In [4]:
from IPython.core.display import display, HTML
display(HTML("<style>div.output_scroll { height: 55em; }</style>"))
%matplotlib notebook
import numpy as np
import sympy as sp
import cxroots as cx
import scipy.special as spsp
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
from IPython.display import Code
from IPython.display import Markdown


# -----------------------------------------------------------------------------------------------------------------

def MID_function(n,m,s0,value_tau):
    
    s = sp.symbols('s')  # define variable s for our problem to be solved
    tau = sp.symbols('tau')  # define variable tau : delay
    a = sp.symbols(["a{:d}".format(i) for i in range(n)], real=True)
    alpha = sp.symbols(["alpha{:d}".format(i) for i in range(m + 1)], real=True)
    Polynomial = s**n + np.array(a).dot([s**i for i in range(n)]) # Revient à faire s^n + a_{n-1}^{n-1}...
    Delayed = np.array(alpha).dot([s**i for i in range(m+1)])*sp.exp(-s*tau) # Revient à faire 
    Q = Polynomial + Delayed 
    SysDerivatif = [Q]
    for i in range(n+m+1):
        DerniereDerivee = SysDerivatif[-1]
        SysDerivatif.append(DerniereDerivee.diff(s)) # Dérivée par rapport à s
    sol = sp.linsolve(SysDerivatif[:-1], alpha + a).args[0] # Solveur selon les alpha et les a
    solNum = sol.subs({s : s0})
    solNum = solNum.subs({tau : value_tau})
    a_num = list(solNum[m + 1:])
    alpha_num = list(solNum[:m + 1])
    QNumerique = s**n + np.array(a_num).dot([s**i for i in range(n)])+\
    np.array(alpha_num).dot([s**i for i in range(m+1)])*sp.exp(-s*tau)
    QNumerique = QNumerique.subs(tau, value_tau)
    sysRootFinding = [QNumerique, QNumerique.diff(s)]
    sysFunc = [sp.lambdify(s, i) for i in sysRootFinding]
    rect = cx.Rectangle([-100, 10], [-100, 100])
    roots = rect.roots(sysFunc[0], sysFunc[1], rootErrTol=1e-5, absTol=1e-5, M = n + m + 1)
    xroot = np.real(roots[0])
    yroot = np.imag(roots[0])
    return xroot,yroot,QNumerique

def factorization_integral_latex(n, m, s0, tau):
    factor = str(tau**(m+1) / spsp.factorial(m))
    parenthesis = "(s + " + str(-s0) + ")"
    power = "^{" + str(n + m + 1) + "}"
    
    return r"\$" + "\Delta(s) = " + factor + parenthesis + power + r"\int_0^1 t^{" + str(m) + r"} (1 - t)^{" + str(n) + "} e^{-" + str(tau) + "t" + parenthesis + "} \mathrm{d}t" + "$"

def factorization_1f1_latex(n, m, s0, tau):
    factor = str(tau**(m+1) * spsp.factorial(n) / spsp.factorial(n+m+1))
    parenthesis = "(s + " + str(-s0) + ")"
    power = "^{" + str(n + m + 1) + "}"
    
    return r"\$" + "\Delta(s) = " + factor + parenthesis + power + r" {}_1 F_1(" + str(m+1) + r", " + str(n+m+2) + ", -" + str(tau) + parenthesis + ")" + "$" 

# -----------------------------------------------------------------------------------------------------------------

output_mid_generic = widgets.Output()
output_mid_control_oriented = widgets.Output()
output_crrid_generic = widgets.Output()

output_root_spectrum_computation_mid_generic = widgets.Output()
output_output_equation_computation_mid_generic = widgets.Output()
output_factorized_integral_equation_computation_mid_generic = widgets.Output()
output_factorization_1f1_equation_computation_mid_generic = widgets.Output()

# -----------------------------------------------------------------------------------------------------------------

slider_n_mid_generic = widgets.IntSlider(min=0,max=3,step=1,description='n :',value=2)
slider_m_mid_generic = widgets.IntSlider(min=0,max=3,step=1,description='m :',value=1)

FloatText_s0_mid_generic = widgets.BoundedFloatText(min=-10.0,max=0.0,step=0.1,description='s0 :',value=-3.0,disabled=False)
FloatText_tau_mid_generic = widgets.BoundedFloatText(min=0.50,max=2.00,step=0.01,description='tau :',value=0.73,disabled=False)

layout_button_ready_to_compute_mid_generic = widgets.Layout( width='100%')
layout_valid_statut_mid_generic = widgets.Layout( width='99%', border='2px solid #8e2042')

button_ready_to_compute_mid_generic = widgets.Button(description='Compute',layout = layout_button_ready_to_compute_mid_generic, disabled=False,button_style='info',icon='check-circle')
valid_statut_mid_generic = widgets.Valid(value=False,description='Status :',layout = layout_valid_statut_mid_generic)

layout_validate_mid_generic =  widgets.TwoByTwoLayout(top_left=button_ready_to_compute_mid_generic,top_right=valid_statut_mid_generic, merge=False)


# -----------------------------------------------------------------------------------------------------------------

def computation_mid_generic(n,m,s0,value_tau):
    
    output_root_spectrum_computation_mid_generic.clear_output()
    output_output_equation_computation_mid_generic.clear_output()
    output_factorized_integral_equation_computation_mid_generic.clear_output()
    output_factorization_1f1_equation_computation_mid_generic.clear_output()
    
    with output_root_spectrum_computation_mid_generic:
        
        fig, ax = plt.subplots(figsize=(6, 4))
        ax.set_title("Root spectrum")
        ax.set_xlabel(r"Re$(s)$")
        ax.set_ylabel(r"Im$(s)$")
        ax.axhline(linewidth=2, color='black', zorder = 2)
        ax.axvline(linewidth=2, color='black', zorder = 2)
        ax.axvspan(0, 1, alpha=0.5, color='red', zorder = 2)
        ax.grid()
        line, = ax.plot([], [], 'o', color='steelblue', zorder = 3)
        
        xr,yr,eq = MID_function(n,m,s0,value_tau)
        line.set_data(xr, yr)
        ax.relim()
        ax.autoscale_view()
        fig.canvas.draw()
    
    with output_output_equation_computation_mid_generic:
        print ("Fp = ",eq)
    
    with output_factorized_integral_equation_computation_mid_generic:
        factorization_integral_latex_equation_mid_generic = factorization_integral_latex(n, m, s0, value_tau)
        display(Markdown(factorization_integral_latex_equation_mid_generic))
    
    with output_factorization_1f1_equation_computation_mid_generic:
        factorization_1f1_latex_equation_mid_generic = factorization_1f1_latex(n, m, s0, value_tau)
        display(Markdown(factorization_1f1_latex_equation_mid_generic))

def button_ready_to_compute_clicked_mid_generic(self):
    
    valid_statut_mid_generic.value = False
    n_mid_generic = slider_n_mid_generic.value
    m_mid_generic = slider_m_mid_generic.value
    s0_mid_generic = FloatText_s0_mid_generic.value
    tau_mid_generic = FloatText_tau_mid_generic.value
    
    computation_mid_generic(n_mid_generic,m_mid_generic,s0_mid_generic,tau_mid_generic)
    
    valid_statut_mid_generic.value = True
    
button_ready_to_compute_mid_generic.on_click(button_ready_to_compute_clicked_mid_generic)

# -----------------------------------------------------------------------------------------------------------------

box_input_mid_generic = widgets.HBox([slider_n_mid_generic, slider_m_mid_generic, FloatText_s0_mid_generic, FloatText_tau_mid_generic])


accordion_mid_generic = widgets.Accordion(children=[output_root_spectrum_computation_mid_generic, output_output_equation_computation_mid_generic, output_factorized_integral_equation_computation_mid_generic, output_factorization_1f1_equation_computation_mid_generic])
accordion_mid_generic.set_title(0, 'Root spectrum')
accordion_mid_generic.set_title(1, 'Output equation')
accordion_mid_generic.set_title(2, 'Factorized integral equation')
accordion_mid_generic.set_title(3, 'Hypergeometric factorization')


description_label_mid_generic = widgets.Label('Insert degree of polynomial n, degree of delay polynomial m, rightmost root s0 and delay tau, then push the button to compute : ')
box_mid_generic = widgets.VBox([description_label_mid_generic, box_input_mid_generic,output_mid_generic, layout_validate_mid_generic, accordion_mid_generic ])

description_label_mid_control_oriented = widgets.Label('Insert degree of polynomial n, degree of delay polynomial m, and rightmost root s0 or delay tau, then push the button to compute : ')
box_mid_control_oriented = widgets.VBox([description_label_mid_control_oriented,output_mid_control_oriented])

description_label_crrid_generic = widgets.Label('Insert degree of polynomial n, degree of delay polynomial m, rightmost root s0 and delay tau, then push the button to compute : ')
box_crrid_generic = widgets.VBox([description_label_crrid_generic,output_crrid_generic])

tab = widgets.Tab([box_mid_generic, box_mid_control_oriented, box_crrid_generic])
tab.set_title(0, 'MID Generic')
tab.set_title(1, 'MID Control oriented')
tab.set_title(2, 'CRRID Generic')

file = open("images/header_p3d_web.png", "rb")
image = file.read()
header_image = widgets.Image(value=image,format='png',width=1980,height=300)

dashboard = widgets.VBox([header_image,tab])
display(dashboard)